### Functions
---

In [ ]:
def printer(iterable:list|dict):
    """
    Displays Lists and Dictionaries nicely
    """
    if isinstance(iterable,list):
        # print("List".center(16,"-"))
        for item in iterable:
            print(f" - {item}")
        print()
    elif isinstance(iterable,dict):
        # print("Dictionary".center(16,"-"))
        
        for key in list(iterable.keys()):
            print(f" {key} :",iterable[key])
        print()
    else:
        print(iterable)


In [46]:
import os

def getClassNames(path = 'processedData/train'):
    folderNames = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
    cleanedNames = []
    for idx in range(len(folderNames)):
        words = folderNames[idx].split()
        words.pop(0)
        words.pop()
        if "-" in words:
            words.pop(words.index("-"))

        out = " ".join(words)
        cleanedNames.append(out)
    classMapping = {}
    for index, value in enumerate(cleanedNames):
        classMapping[index] = value
    print(classMapping)

    return classMapping

In [47]:
def giveMeImageSize(modelVersion = "B4") -> tuple:
    """
    Returns the resolution for the respective EfficientNet model verisons:

    "B0", "B1", "B2", "B3", "B4", "B"5, "B6", "B7"

    """
    if modelVersion == "B0":
        return (224,224)
    elif modelVersion == "B1":
        return (240,240)
    elif modelVersion == "B2":
        return (260,260)
    elif modelVersion == "B3":
        return (300,300)
    elif modelVersion == "B4":
        return (380,380)
    elif modelVersion == "B5":
        return (456,456)
    elif modelVersion == "B6":
        return (528,528)
    elif modelVersion == "B7":
        return (600,600)


In [48]:
from torchvision import transforms
#? function for image transformation
def pad_to_Square(image):
    width, height = image.size
    max_side = max(width,height)
    left_pad = (max_side-width)//2
    right_pad = (max_side-width-left_pad)
    top_pad = (max_side-height)//2
    bot_pad = (max_side-height - top_pad)

    padded_image = transforms.functional.pad(image, (left_pad, top_pad, right_pad, bot_pad), padding_mode='constant', fill=0)
    return padded_image


In [49]:
import os
from sklearn.model_selection import train_test_split
from PIL import Image
# Function to create the necessary directories
def create_dirs(base_dir, classes):
    os.makedirs(base_dir, exist_ok=True)
    for subset in ['train', 'valid', 'test']:
        subset_path = os.path.join(base_dir, subset)
        os.makedirs(subset_path, exist_ok=True)
        for class_name in classes:
            os.makedirs(os.path.join(subset_path, class_name), exist_ok=True)

# Function to split and save images into train and test directories
def split_and_process_images(raw_data_dir:str, processed_data_dir:str, randomState=42):
    # Get all class names (subfolder names)
    classes = os.listdir(raw_data_dir)
    print("Subfolders:",classes)
    create_dirs(processed_data_dir, classes)
    
    # Iterate through each class folder
    for class_name in classes:
        class_folder = os.path.join(raw_data_dir, class_name)
        if os.path.isdir(class_folder):
            # Get all image filenames
            image_filenames = os.listdir(class_folder)
            print(f"{class_name} : {len(image_filenames)} files.")
            # Split into train and test sets
            train_files, testval_files = train_test_split(image_filenames, test_size=0.2, random_state=randomState)
            print(f" Train: {len(train_files)}, Test&Val: {len(testval_files)}")
            # Process and save training images
            for filename in train_files:
                image_path = os.path.join(class_folder, filename)
                img = Image.open(image_path)
                # Save the training image
                train_save_path = os.path.join(processed_data_dir, 'train', class_name, filename)
                img.save(train_save_path)

            # Split into validation and test sets
            val_files, test_files = train_test_split(testval_files, test_size=0.5, random_state=randomState)

            # Process and save validation images
            for filename in val_files:
                image_path = os.path.join(class_folder, filename)
                img = Image.open(image_path)
                # Save the validation image
                val_save_path = os.path.join(processed_data_dir, 'valid', class_name, filename)
                img.save(val_save_path)

            # Process and save test images
            for filename in test_files:
                image_path = os.path.join(class_folder, filename)
                img = Image.open(image_path)
                # Save the testing image
                test_save_path = os.path.join(processed_data_dir, 'test', class_name, filename)
                img.save(test_save_path)

        else:
            print(f"Error on {class_name}")


In [50]:
def showGrads(model):
    for name, param in model.named_parameters():
        print(f"Layer: {name}".ljust(46), f"requires_grad: {param.requires_grad}")

def freeze_model(model):
    for params in model.parameters():
        params.requires_grad=False

def unfreeze_model(model):
    for params in model.parameters():
        params.requires_grad = True 
    
def unfreeze_last_n_blocks(model,n):
    lastblock = len(model.features)-1
    blocknames = ["classifier"] + [f"features.{lastblock-i}" for i in range(n)]
    for name,params in model.named_parameters():
        if any(substring in name for substring in blocknames): 
            params.requires_grad = True

    print(f"Layers unfrozen: {blocknames}")


### Data Loaders and Processing
---

#### Cleaning folder names into class names

In [37]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [38]:
imgSize = giveMeImageSize()
print(imgSize)

(380, 380)


#### Getting Mean and STD for image normalization
---

In [18]:
collectMean = False  #? <- Update Mean and STD if model is chagned or image is resized


if collectMean:
    #? Getting the mean and std of the raw dataset
    initial_Transform = transforms.Compose([
        transforms.Lambda(pad_to_Square),
        transforms.Resize(imgSize),  # Resize images
        transforms.ToTensor(),  # Convert images to tensor
    ])

    #? raw dataset File path
    raw_path = './data'

    # Load raw dataset
    dataset = datasets.ImageFolder(root=raw_path, transform=initial_Transform)
    loader = DataLoader(dataset, batch_size=16, shuffle=False)       #? num_workers indicate the number of parallel processes

    # Initialize sums
    mean = 0.
    std = 0.
    total_images = 0

    print("dataset:", dataset)
    batch = 0
    print(f"Total Batches: {len(loader)}")
    for images, _ in loader:
        batch_samples = images.size(0)  # batch size
        images = images.view(batch_samples, images.size(1), -1)  # flatten H and W
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += batch_samples
        batch += 1
        print(f"Batch [{batch}/{len(loader)}]".ljust(20),
            f"Mean: {(mean/total_images)}",
            f"STD: {(std/total_images)}", end="\r")
    print()

    mean /= total_images
    std /= total_images

    print(f"Mean: {mean}")
    print(f"Std: {std}")

#### Load Datasets:
- Training DataSet with augmentation.
- Testing DataSet without augmentation.
---


In [39]:
 #? Copy form previous cell outputs
mean = torch.tensor([0.5246, 0.3975, 0.3844])
std = torch.tensor([0.2752, 0.2196, 0.2174])

transform_train = transforms.Compose([
    transforms.Lambda(pad_to_Square),
    transforms.Resize(imgSize),  # Resize images
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=mean, std=std),  # Normalize images
])

transform_test = transforms.Compose([
    transforms.Lambda(pad_to_Square),
    transforms.Resize(imgSize),  # Resize images
    transforms.ToTensor(),  # Convert images to tensor
])


In [40]:

#? training dataset directory
train_path = "processedData/train"
#? validation dataset directory
valid_path = "processedData/valid"
#? testing dataset directory
test_path = "processedData/test"

batchSize = 64
if train_path:
    #? Load TRAIN dataset from directory
    train_dataset = datasets.ImageFolder(root=train_path, transform=transform_train)
    #? Create a DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
    print(train_dataset, f"Batches: {len(train_dataloader)}","\n".ljust(50,"-"))

if valid_path:
    #? Load VALID dataset from directory
    valid_dataset = datasets.ImageFolder(root=valid_path, transform=transform_test)
    #? Create a DataLoader
    valid_dataloader = DataLoader(valid_dataset, batch_size=batchSize, shuffle=False)
    print(valid_dataset, f"Batches: {len(valid_dataloader)}","\n".ljust(50,"-"))

if test_path:
    #? Load TEST dataset from directory
    test_dataset = datasets.ImageFolder(root=test_path, transform=transform_test)
    #? Create a DataLoader
    test_dataloader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)
    print(test_dataset, f"Batches: {len(test_dataloader)}","\n".ljust(50,"-"))

Dataset ImageFolder
    Number of datapoints: 21719
    Root location: processedData/train
    StandardTransform
Transform: Compose(
               Lambda()
               Resize(size=(380, 380), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=tensor([0.5246, 0.3975, 0.3844]), std=tensor([0.2752, 0.2196, 0.2174]))
           ) Batches: 340 
-------------------------------------------------
Dataset ImageFolder
    Number of datapoints: 2715
    Root location: processedData/valid
    StandardTransform
Transform: Compose(
               Lambda()
               Resize(size=(380, 380), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           ) Batches: 43 
-------------------------------------------------
Dataset ImageFolder
    Number of datapoints: 2719
    Root location: processedData/test
    StandardTransform
Transform: Compose(
               Lambda()
               Resize(size=(380, 380)

### Model Training
---

##### Model
---

In [41]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

efficientnetmodel = models.efficientnet_b4(pretrained=True)

no_features = efficientnetmodel.classifier[1].in_features  
efficientnetmodel.classifier[1] = nn.Linear(no_features, 10) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnetmodel = efficientnetmodel.to(device)

criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(efficientnetmodel.parameters(), lr=0.005)

freeze_model(efficientnetmodel)
showGrads(efficientnetmodel)

Layer: features.0.0.weight                     requires_grad: False
Layer: features.0.1.weight                     requires_grad: False
Layer: features.0.1.bias                       requires_grad: False
Layer: features.1.0.block.0.0.weight           requires_grad: False
Layer: features.1.0.block.0.1.weight           requires_grad: False
Layer: features.1.0.block.0.1.bias             requires_grad: False
Layer: features.1.0.block.1.fc1.weight         requires_grad: False
Layer: features.1.0.block.1.fc1.bias           requires_grad: False
Layer: features.1.0.block.1.fc2.weight         requires_grad: False
Layer: features.1.0.block.1.fc2.bias           requires_grad: False
Layer: features.1.0.block.2.0.weight           requires_grad: False
Layer: features.1.0.block.2.1.weight           requires_grad: False
Layer: features.1.0.block.2.1.bias             requires_grad: False
Layer: features.1.1.block.0.0.weight           requires_grad: False
Layer: features.1.1.block.0.1.weight           r

#### Training from Start
---

In [ ]:
# import pickle
from datetime import datetime
modelSaver_path = f"modelLogs/{datetime.today().strftime('%Y-%m-%d %H-%M')}"
print("Save path:", modelSaver_path)

epochsPerPhase = 5
phases = 4                          #? <-- this determines number of blocks to unfreeze
modelVariation = "type2Phases"      #? <-- Change this each time you run a new Model or training type

for phase in range(phases):
    print(f"Phase: {phase+1}") 
    #? progressive unfreezing of layers
    unfreeze_last_n_blocks(efficientnetmodel, phase)

    efficientnetmodel.train()
    for epoch in range(epochsPerPhase):

        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for batch, (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimiser.zero_grad()
            outputs = efficientnetmodel(inputs).squeeze()
        
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs,1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

            print(f" Epoch [{epoch+1}/{epochsPerPhase}]".ljust(16),
                  f"Batch: {batch+1}".ljust(10),
                  f"Accuracy: {((correct_train / total_train)*100):.2f}%".ljust(20),
                  end="\r")
            
        
        train_accuracy = (correct_train / total_train)*100
        avg_loss = running_loss / len(train_dataloader)

        print(f" Epoch [{epoch+1}/{epochsPerPhase}]".ljust(16),
              f"Loss: {avg_loss:.4f}".ljust(10),
              f"Accuracy: {train_accuracy:.2f}%".ljust(30))

        os.makedirs(modelSaver_path,exist_ok=True)
        checkpoint_filename = modelSaver_path+f"/{modelVariation}_phase{phase+1}_epoch{epoch+1}.pth"
        torch.save(efficientnetmodel.state_dict(), checkpoint_filename)
        print(f"model saved to {checkpoint_filename}")

                
    print(" Evaluation ".center(30,"-"))
    efficientnetmodel.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for vBatch, (inputs, labels) in enumerate(valid_dataloader):
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = efficientnetmodel(inputs).squeeze()
            _, predicted = torch.max(outputs,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            print(f"Epoch: {epoch+1}".ljust(10), f"Batch: {vBatch+1}".ljust(12), f"Accuracy: {((correct/total)*100):.2f}%",end="\r")
            

    val_accuracy = (correct / total)*100
    print(f" Validation Accuracy: {val_accuracy:.2f}%".ljust(50))
    print()

#### Load Model From Saver
---

In [42]:
import torch

modelSaver_path = "modelLogs/2025-03-28 02-15"
modelVariation = "type2Phases"
checkpoint = torch.load(f"{modelSaver_path}/{modelVariation}_phase4_epoch1.pth",weights_only=False)

for key in list(checkpoint.keys()):
    print(key)


efficientnetmodel.load_state_dict(checkpoint)

features.0.0.weight
features.0.1.weight
features.0.1.bias
features.0.1.running_mean
features.0.1.running_var
features.0.1.num_batches_tracked
features.1.0.block.0.0.weight
features.1.0.block.0.1.weight
features.1.0.block.0.1.bias
features.1.0.block.0.1.running_mean
features.1.0.block.0.1.running_var
features.1.0.block.0.1.num_batches_tracked
features.1.0.block.1.fc1.weight
features.1.0.block.1.fc1.bias
features.1.0.block.1.fc2.weight
features.1.0.block.1.fc2.bias
features.1.0.block.2.0.weight
features.1.0.block.2.1.weight
features.1.0.block.2.1.bias
features.1.0.block.2.1.running_mean
features.1.0.block.2.1.running_var
features.1.0.block.2.1.num_batches_tracked
features.1.1.block.0.0.weight
features.1.1.block.0.1.weight
features.1.1.block.0.1.bias
features.1.1.block.0.1.running_mean
features.1.1.block.0.1.running_var
features.1.1.block.0.1.num_batches_tracked
features.1.1.block.1.fc1.weight
features.1.1.block.1.fc1.bias
features.1.1.block.1.fc2.weight
features.1.1.block.1.fc2.bias
feat

<All keys matched successfully>

#### Train From Saver
---

In [ ]:

epochsPerPhase = 5
resumePhase = 4  #? <- refer to saveFile
phases = 5


for phase in range(resumePhase,phases):
    print(f"Phase: {phase+1}") 
    #? progressive unfreezing of layers
    unfreeze_last_n_blocks(efficientnetmodel, phase)

    efficientnetmodel.train()
    for epoch in range(epochsPerPhase):

        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for batch, (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimiser.zero_grad()
            outputs = efficientnetmodel(inputs).squeeze()
        
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs,1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

            print(f" Epoch [{epoch+1}/{epochsPerPhase}]".ljust(16),
                  f"Batch: {batch+1}".ljust(10),
                  f"Accuracy: {((correct_train / total_train)*100):.2f}%".ljust(20),
                  end="\r")
            
        
        train_accuracy = (correct_train / total_train)*100
        avg_loss = running_loss / len(train_dataloader)

        print(f" Epoch [{epoch+1}/{epochsPerPhase}]".ljust(16),
              f"Loss: {avg_loss:.4f}".ljust(10),
              f"Accuracy: {train_accuracy:.2f}%".ljust(30))

        os.makedirs(modelSaver_path,exist_ok=True)
        checkpoint_filename = modelSaver_path+f"/{modelVariation}_phase{phase+1}_epoch{epoch+1}.pth"
        torch.save(efficientnetmodel.state_dict(), checkpoint_filename)
        print(f"model saved to {checkpoint_filename}")

                
    print(" Evaluation ".center(30,"-"))
    efficientnetmodel.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for vBatch, (inputs, labels) in enumerate(valid_dataloader):
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = efficientnetmodel(inputs).squeeze()
            _, predicted = torch.max(outputs,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            print(f"Epoch: {epoch+1}".ljust(10), f"Batch: {vBatch+1}".ljust(12), f"Accuracy: {((correct/total)*100):.2f}%",end="\r")
            

    val_accuracy = (correct / total)*100
    print(f" Validation Accuracy: {val_accuracy:.2f}%".ljust(50))
    print()

Phase: 5
Layers unfrozen: ['classifier', 'features.8', 'features.7', 'features.6', 'features.5']


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.56 GiB is allocated by PyTorch, and 216.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#### Test Model
---

In [43]:
print(" Testing ".center(30,"-"))
efficientnetmodel.eval()
correct = 0
total = 0

with torch.no_grad():
    for tBatch, (inputs, labels) in enumerate(test_dataloader):
        inputs, labels = inputs.to(device), labels.float().to(device)
        outputs = efficientnetmodel(inputs).squeeze()
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print(f"Batch: {tBatch+1}".ljust(12), f"Accuracy: {((correct/total)*100):.2f}%",end="\r")
        

val_accuracy = (correct / total)*100
print(f" Validation Accuracy: {val_accuracy:.2f}%".ljust(50))
print()

---------- Testing -----------
 Validation Accuracy: 71.20%                      

